# Example 1.1
This example follows the `examples/1.1`  
Search for a simple lj crystal with 8 numbers percell

In [1]:
from airsspy import SeedAtoms, Buildcell
from ipypb import ipb as tqdm

# Import ase assets
from ase.optimize import BFGSLineSearch, sciopt, FIRE, BFGS, precon
from ase.calculators.lj import LennardJones
from ase.constraints import UnitCellFilter, ExpCellFilter

In [2]:
# Define the search seed

seed = SeedAtoms('Al', cell=[2, 2, 2], pbc=True)
seed.gentags.minsep = 1.5
# Define per-atom tags, here it is simply request 8 Al atoms
al = seed[0]
al.num = 8

In [3]:
# LJ potential calculator
lj = LennardJones(signma=2, epsilon=1, beta=1, rc=2.5)

In [4]:
# The seed in text form
print('\n'.join(seed.get_cell_inp_lines()))

%BLOCK lattice_cart
2.0000000000  0.0000000000  0.0000000000
0.0000000000  2.0000000000  0.0000000000
0.0000000000  0.0000000000  2.0000000000
%ENDBLOCK lattice_cart
%BLOCK positions_abs
Al  0.0000000000 0.0000000000 0.0000000000 # Al0 % NUM=8
%ENDBLOCK positions_abs
#MINSEP=1.5



## Generate the random structure
The random structrue can be built by calling `build_random_atoms` method.
An `ase.Atoms` object is returned.

In [5]:
rand_atoms = seed.build_random_atoms()

Inpsect the cell

In [6]:
rand_atoms.get_cell()

array([[2.35951   , 0.        , 0.        ],
       [0.48072515, 4.08875698, 0.        ],
       [0.97720172, 0.51659301, 6.32963116]])

Inspect the cell volume
The initial cell volume is $2\times2\times2=8$ and we requested 8 atoms.
So the target cell volume is 64 but `buildcell` will scale to a random value
within 50%.

In [7]:
rand_atoms.get_volume()

61.064882281705266

In [8]:
rand_atoms.get_positions()

array([[0.37593082, 1.38684595, 2.64249205],
       [0.10799053, 3.54577896, 1.73390679],
       [2.27238155, 3.40699766, 5.63543925],
       [1.45917171, 3.17753425, 1.57839886],
       [1.07000499, 2.14481009, 5.94208792],
       [1.67583419, 1.86035924, 2.2477568 ],
       [0.86334848, 3.62557623, 5.97221832],
       [0.63123853, 1.74727672, 0.86564539]])

## Do the search
The search can be carried out directly using the calculator and optimizers in `ase` 

In [9]:
def gen_and_relax(seed, calc, opti):
    """
    Generate and relax the cell
    """
    rand_atoms = seed.build_random_atoms()
    rand_atoms.set_pbc(True)  # Set the PBC as we are search from crystal
    rand_atoms.set_calculator(calc)  # Attached hte calculator
    # Apply the UnitCellFilter as we are also optimizing the cell
    opt = opti(UnitCellFilter(rand_atoms), logfile=None) 
    opt.run(fmax=0.05)
    return rand_atoms

In [11]:
res = []
for i in tqdm(range(20)):
    # A range of the possible optimization algorithm are possible
    relaxed = gen_and_relax(seed, lj, BFGSLineSearch)
    res.append(relaxed)

 [████████████████████████████████████████████████████████████] 20/20 [00:38<00:01, 1.90s/it]

## Get the symmetry of the relaxed structures

In [12]:
from spglib import get_spacegroup
spg = [get_spacegroup(i, 0.5) for i in res]
spg

['Fm-3m (225)',
 'Fm-3m (225)',
 'Fm-3m (225)',
 'P2_1/m (11)',
 'Fm-3m (225)',
 'Cm (8)',
 'P6_3/mmc (194)',
 'Fm-3m (225)',
 'P6_3/mmc (194)',
 'C2/m (12)',
 'Fm-3m (225)',
 'Fm-3m (225)',
 'Fm-3m (225)',
 'Imma (74)',
 'Cm (8)',
 'C2/m (12)',
 'Fm-3m (225)',
 'P6_3/mmc (194)',
 'P-1 (2)',
 'P6_3/mmc (194)']

## Alternatively, use the native airss script to run the search

In [ ]:
!mkdir work

In [ ]:
%%timeit -n 1 -r 1
seed.write_seed('work/Al.cell')
!echo "1 12 6 2.5\nAl\n# Epsilon\n1\n# Sigma\n2\n# Beta\n1" >> work/Al.pp
!cd work && airss.pl -seed Al -max 20 -pp3 && cd ../

Note that the `pp3` program for relxation will run significantly faster than doing so in python

In [ ]:
!cd work/ &&  ca -r -u 0.01 && cd -